In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from sklearn.metrics import mean_squared_error

In [ ]:
# Load MovieLens Dataset
start_time = time.time()
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

In [ ]:
# Convert timestamps to human-readable format
ratings["date"] = pd.to_datetime(ratings["timestamp"], unit='s')
ratings.drop(columns=["timestamp"], inplace=True)

In [ ]:
# Handle missing values
ratings.dropna(inplace=True)

In [ ]:
# Filter out cold-start users and movies
movie_counts = ratings["movieId"].value_counts()
user_counts = ratings["userId"].value_counts()
ratings = ratings[ratings["movieId"].isin(movie_counts[movie_counts >= 10].index)]
ratings = ratings[ratings["userId"].isin(user_counts[user_counts >= 10].index)]

In [ ]:
# Normalize ratings
min_rating = ratings["rating"].min()
max_rating = ratings["rating"].max()
ratings["normalized_rating"] = (ratings["rating"] - min_rating) / (max_rating - min_rating)

In [ ]:
# Train NCF Model
start_time = time.time()
num_users = ratings["userId"].nunique()
num_movies = ratings["movieId"].nunique()

user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=num_users, output_dim=50)(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=50)(movie_input)

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_vec, movie_vec])
output = Dense(1, activation='linear')(dot_product)

ncf_model = Model([user_input, movie_input], output)
ncf_model.compile(optimizer='adam', loss='mse')

# Generate training data
train_users = np.random.randint(0, num_users, size=(100000,))
train_movies = np.random.randint(0, num_movies, size=(100000,))
train_ratings = np.random.rand(100000)

ncf_model.fit([train_users, train_movies], train_ratings, epochs=10, batch_size=64)
end_time = time.time()
print(f"NCF Model Training Time: {end_time - start_time:.4f} seconds")

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 0.1713
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 0.0602
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 27s 18ms/step - loss: 0.0094
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 27s 18ms/step - loss: 0.0036
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 0.0049
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 27s 18ms/step - loss: 0.0070
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 0.0051
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 0.0037
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 0.0039
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - loss: 0.0040
NCF Model Training Time: 276.5037 seconds


In [ ]:
# Evaluate NCF Model
start_time = time.time()
predicted_ratings = ncf_model.predict([train_users, train_movies])
mse_ncf = mean_squared_error(train_ratings, predicted_ratings)
rmse_ncf = np.sqrt(mse_ncf)
end_time = time.time()
print(f"Model Evaluation Time: {end_time - start_time:.4f} seconds")
print(f"NCF Model -> RMSE: {rmse_ncf}, MSE: {mse_ncf}")

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 3s 969us/step
Model Evaluation Time: 4.2687 seconds
NCF Model -> RMSE: 0.06258981040117298, MSE: 0.003917484366054782
